In [1]:
%load_ext autoreload
%autoreload 1

In [36]:
# Import libraries.
# pip install requests
import requests

# pip isntall pandas
import pandas as pd

# Import the API key.
from config import census_api_key

In [37]:
# Year and State for API pulls (this could change to user input fields)

#user_input_year = input('Enter year (current is 2018):')
year_value = '2018'

# Need to create key:value for * so this can be run for all states
#user_input_state = input('Enter State (enter * for all states):')
state_value = 'California'

In [38]:
# Pull census state data from API
# Use try except to confirm response is 200 and not 404
# OR use if code == 200 then data found else data not found
states_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?get=NAME&for=state:*&key={census_api_key}'
print(states_api)

states_list = requests.get(states_api)
print(states_list)

# Create list from states API pull
states_list = states_list.json()
print(states_list)

https://api.census.gov/data/2018/acs/acs5/profile?get=NAME&for=state:*&key=c4922650af1ec523155c3c6aec47c5491288a40c
<Response [200]>
[['NAME', 'state'], ['Minnesota', '27'], ['Mississippi', '28'], ['Missouri', '29'], ['Montana', '30'], ['Nebraska', '31'], ['Nevada', '32'], ['New Hampshire', '33'], ['New Jersey', '34'], ['New Mexico', '35'], ['New York', '36'], ['North Carolina', '37'], ['North Dakota', '38'], ['Ohio', '39'], ['Oklahoma', '40'], ['Oregon', '41'], ['Pennsylvania', '42'], ['Rhode Island', '44'], ['South Carolina', '45'], ['South Dakota', '46'], ['Tennessee', '47'], ['Texas', '48'], ['Utah', '49'], ['Vermont', '50'], ['Virginia', '51'], ['West Virginia', '54'], ['Washington', '53'], ['Wisconsin', '55'], ['Wyoming', '56'], ['Puerto Rico', '72'], ['Alabama', '01'], ['Alaska', '02'], ['Arizona', '04'], ['Arkansas', '05'], ['California', '06'], ['Colorado', '08'], ['Connecticut', '09'], ['Delaware', '10'], ['District of Columbia', '11'], ['Florida', '12'], ['Georgia', '13'], [

In [39]:
# Rename index [0] sublist for better header discriptions
for index, sublist in enumerate(states_list):
    if sublist[0] == 'NAME':
        states_list[index][0] = 'state_name'
        states_list[index][1] = 'state_id'

# Create DataFrame for database
states_df = pd.DataFrame(states_list)



In [40]:
# Make index 0 header, reorder columns, and sort columns
# Reference: https://stackoverflow.com/questions/31328861/python-pandas-replacing-header-with-top-row
new_header = states_df.iloc[0]
states_df = states_df[1:]
states_df.columns = new_header
states_df = states_df.reindex(columns=['state_id','state_name'])

# Sort by state_id
#Sort_values is not working, tried changing type to float or int, but still does not work.
states_df["state_id"] = states_df.state_id.astype(int)
#states_df.sort_values(by=['state_id'])
#states_df.dtypes

states_df.head()

# Export states as CSV
states_df.to_csv('resources/states.csv', index=False) 

,state_id,state_name
1,27,Minnesota
2,28,Mississippi
3,29,Missouri
4,30,Montana
5,31,Nebraska


In [11]:
# Create dictionary for states_list to use for other API calls
states_dict = {}
for l2 in states_list:
    key, value = l2
    states_dict[key] = value

print(states_dict)

{'state_name': 'state_id', 'Minnesota': '27', 'Mississippi': '28', 'Missouri': '29', 'Montana': '30', 'Nebraska': '31', 'Nevada': '32', 'New Hampshire': '33', 'New Jersey': '34', 'New Mexico': '35', 'New York': '36', 'North Carolina': '37', 'North Dakota': '38', 'Ohio': '39', 'Oklahoma': '40', 'Oregon': '41', 'Pennsylvania': '42', 'Rhode Island': '44', 'South Carolina': '45', 'South Dakota': '46', 'Tennessee': '47', 'Texas': '48', 'Utah': '49', 'Vermont': '50', 'Virginia': '51', 'West Virginia': '54', 'Washington': '53', 'Wisconsin': '55', 'Wyoming': '56', 'Puerto Rico': '72', 'Alabama': '01', 'Alaska': '02', 'Arizona': '04', 'Arkansas': '05', 'California': '06', 'Colorado': '08', 'Connecticut': '09', 'Delaware': '10', 'District of Columbia': '11', 'Florida': '12', 'Georgia': '13', 'Idaho': '16', 'Hawaii': '15', 'Illinois': '17', 'Indiana': '18', 'Iowa': '19', 'Kansas': '20', 'Kentucky': '21', 'Louisiana': '22', 'Maine': '23', 'Maryland': '24', 'Massachusetts': '25', 'Michigan': '26'}


In [51]:
# Print state code from state_value (or user input)
#states_dict[user_input_state]
states_dict[state_value]

'06'

In [52]:
# Pull census demographic data from API
census__population_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP05_0001E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_population_list = requests.get(census__population_api)
census_population_list = census_population_list.json()

census_internet_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP02_0150E,DP02_0151E,DP02_0152E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_internet_list = requests.get(census_internet_api)
census_internet_list = census_internet_list.json()

census_age_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP05_0005E,DP05_0006E,DP05_0007E,DP05_0008E,DP05_0009E,DP05_0010E,DP05_0011E,DP05_0012E,DP05_0013E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_age_list = requests.get(census_age_api)
census_age_list = census_age_list.json()

census_sex_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP05_0002E,DP05_0003E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_sex_list = requests.get(census_sex_api)
census_sex_list = census_sex_list.json()

census_income_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP03_0051E,DP03_0052E,DP03_0053E,DP03_0054E,DP03_0055E,DP03_0056E,DP03_0057E,DP03_0058E,DP03_0059E,DP03_0060E,DP03_0061E,DP03_0062E,DP03_0063E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_income_list = requests.get(census_income_api)
census_income_list = census_income_list.json()

census_education_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP02_0059PE,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_education_list = requests.get(census_education_api)
census_education_list = census_education_list.json()

census_race_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP05_0070E,DP05_0071E,DP05_0076E,DP05_0079E,DP05_0080E,DP05_0078E,DP05_0081E,DP05_0082E,DP05_0083E,DP05_0085E,DP05_0084E,DP05_0077E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_race_list = requests.get(census_race_api)
census_race_list = census_race_list.json()

census_citizenship_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP02_0086E,DP02_0087E,DP02_0092E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_citizenship_list = requests.get(census_citizenship_api)
census_citizenship_list = census_citizenship_list.json()

census_disability_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP02_0072E,DP02_0074E,DP02_0076E,DP02_0073E,DP02_0075E,DP02_0077E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_disability_list = requests.get(census_disability_api)
census_disability_list = census_disability_list.json()

census_employment_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP03_0003E,DP03_0004E,DP03_0005E,DP03_0007E,DP03_0009PE&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_employment_list = requests.get(census_employment_api)
census_employment_list = census_employment_list.json()

census_occupation_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP03_0026E,DP03_0027E,DP03_0028E,DP03_0029E,DP03_0030E,DP03_0031E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_occupation_list = requests.get(census_occupation_api)
census_occupation_list = census_occupation_list.json()

census_industry_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP03_0033E,DP03_0034E,DP03_0035E,DP03_0036E,DP03_0037E,DP03_0038E,DP03_0039E,DP03_0040E,DP03_0041E,DP03_0042E,DP03_0043E,DP03_0044E,DP03_0045E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_industry_list = requests.get(census_industry_api)
census_industry_list = census_industry_list.json()

census_family_structure_api = f'https://api.census.gov/data/{year_value}/acs/acs5/profile?&get=GEO_ID,NAME,DP02_0001E,DP02_0013E&for=county:*&in=state:{states_dict[state_value]}&key={census_api_key}'
census_family_structure_list = requests.get(census_family_structure_api)
census_family_structure_list = census_family_structure_list.json()


In [53]:
print(census__population_api)

https://api.census.gov/data/2018/acs/acs5/profile?&get=GEO_ID,NAME,DP05_0001E&for=county:*&in=state:06&key=c4922650af1ec523155c3c6aec47c5491288a40c


In [15]:
# Rename header columns and create DataFrame
for index, sublist in enumerate(census_list):
    if sublist[0] == 'GEO_ID':
        census_list[index][0] = 'geo_id'
        census_list[index][1] = 'county_name'
        census_list[index][2] = 'household_population'
        census_list[index][3] = 'household_with_income'
        census_list[index][4] = 'household_with_income_less_than_10000'
        census_list[index][5] = 'household_with_income_between_10000_to_14999'
        census_list[index][6] = 'household_with_income_between_15000_to_24999'
        census_list[index][7] = 'household_with_income_between_25000_to_34999'
        census_list[index][8] = 'household_with_income_between_35000_to_49999'
        census_list[index][9] = 'household_with_income_between_50000to_74999'
        census_list[index][10] = 'household_with_income_between_75000_to_99999'
        census_list[index][11] = 'household_with_income_between_100000_to_149999'
        census_list[index][12] = 'household_with_income_between_150000_to_199999'
        census_list[index][13] = 'household_with_income_between_greater_than_200000'
        census_list[index][14] = 'household_median_income'
        census_list[index][15] = 'household_mean_income'
        census_list[index][16] = 'median_age'
        census_list[index][17] = 'percent_high_school_graduate_or_higher'
        census_list[index][18] = 'percent_bachelor_degree_or_higher'
        DP02_0065E
        census_list[index][19] = 'state_id'
        census_list[index][20] = 'county_id'
census_df = pd.DataFrame(census_list)
census_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,geo_id,county_name,household_population,household_with_income,household_with_income_less_than_10000,household_with_income_between_10000_to_14999,household_with_income_between_15000_to_24999,household_with_income_between_25000_to_34999,household_with_income_between_35000_to_49999,household_with_income_between_50000to_74999,...,household_with_income_between_100000_to_149999,household_with_income_between_150000_to_199999,household_with_income_between_greater_than_200000,household_median_income,household_mean_income,median_age,percent_high_school_graduate_or_higher,percent_bachelor_degree_or_higher,state_id,county_id
1,0500000US06033,"Lake County, California",63151,25966,2374,2051,3203,3247,3607,4030,...,2609,959,708,42475,60847,1375,85.2,15.8,06,033
2,0500000US06043,"Mariposa County, California",16431,7700,494,484,934,711,1133,1465,...,873,384,162,51199,64590,356,90.4,23.4,06,043
3,0500000US06115,"Yuba County, California",74094,26179,1289,1879,2835,2497,3956,4744,...,3591,1450,674,52624,67391,1117,82.0,16.7,06,115
4,0500000US06013,"Contra Costa County, California",1123209,392277,13870,11757,21416,22514,34834,54684,...,74344,43088,67295,93712,128079,21680,89.4,41.7,06,013


In [16]:
# Make index 0 header, and remove state from county column
new_header = census_df.iloc[0]
census_df = census_df[1:]
census_df.columns = new_header
census_df.head()

,geo_id,county_name,household_population,household_with_income,household_with_income_less_than_10000,household_with_income_between_10000_to_14999,household_with_income_between_15000_to_24999,household_with_income_between_25000_to_34999,household_with_income_between_35000_to_49999,household_with_income_between_50000to_74999,...,household_with_income_between_100000_to_149999,household_with_income_between_150000_to_199999,household_with_income_between_greater_than_200000,household_median_income,household_mean_income,median_age,percent_high_school_graduate_or_higher,percent_bachelor_degree_or_higher,state_id,county_id
1,0500000US06033,"Lake County, California",63151,25966,2374,2051,3203,3247,3607,4030,...,2609,959,708,42475,60847,1375,85.2,15.8,06,033
2,0500000US06043,"Mariposa County, California",16431,7700,494,484,934,711,1133,1465,...,873,384,162,51199,64590,356,90.4,23.4,06,043
3,0500000US06115,"Yuba County, California",74094,26179,1289,1879,2835,2497,3956,4744,...,3591,1450,674,52624,67391,1117,82.0,16.7,06,115
4,0500000US06013,"Contra Costa County, California",1123209,392277,13870,11757,21416,22514,34834,54684,...,74344,43088,67295,93712,128079,21680,89.4,41.7,06,013
5,0500000US06035,"Lassen County, California",21123,9493,497,609,941,909,1150,1979,...,1388,599,250,56362,71097,403,82.8,12.0,06,035


In [17]:
# Export census as CSV
census_df.to_csv('resources/census_data.csv', index=False) 